In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from collections import Counter

In [2]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    
    def is_leaf(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, min_split = 2, max_depth = 100, n_feat = None):
        self.min_split = min_split
        self.max_depth = max_depth
        self.num_feat = n_feat
        self.root = None

    def fit(self, X, y):
        
        self.num_feat = X.shape[1] if not self.num_feat else min(X.shape[1], self.num_feat)
        self.root = self._make_tree(X, y)
    
    def _make_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        if(depth >= self.max_depth or n_labels == 1 or n_samples < self.min_split):
            leaf_value = self._common_label(y)
            return Node(value=leaf_value)
        
        features = np.random.choice(n_features, self.num_feat, replace=False)

        best_features, best_threshold = self._best_split(X, y, features)

        left_idxs, right_idxs = self._split(X[:, best_features], best_threshold)

        left = self._make_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._make_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_features, best_threshold, left, right)
    
    def _best_split(self, X, y, features):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in features:
            X_column = X[:, feat_idx]
            threshold = np.unique(X_column)

            for thr in threshold:
                gain = self._IG(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr
        
        return split_idx, split_threshold
    
    def _IG(self, y, X_column, threshold):
        p_entropy = self._entropy(y)

        left_idx, right_idx = self._split(X_column, threshold)
        
        if len(left_idx) == 0 or len(right_idx) == 0:
            return 0
        
        n = len(y)
        n_l, n_r = len(left_idx), len(right_idx)
        e_l, e_r = self._entropy(y[left_idx]), self._entropy(y[right_idx])
        c_entropy = (n_l/n)* e_l + (n_r/n) * e_r

        information_gain = p_entropy - c_entropy
        return information_gain
    
    def _split(self, X_column, split_tresh):
        left_idxs = np.argwhere(X_column <= split_tresh).flatten()
        right_idxs = np.argwhere(X_column > split_tresh).flatten()
        return left_idxs, right_idxs
    
    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)

        return -np.sum([p*np.log(p) for p in ps if p > 0])
    
    def _common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    def predict(self, X):
        return np.array([self._traverse(x, self.root) for x in X])
    
    def _traverse(self, X, node):
        if node.is_leaf():
            return node.value
        
        if X[node.feature] <= node.threshold:
            return self._traverse(X, node.left)
        return self._traverse(X, node.right)

In [4]:
data = datasets.load_breast_cancer()

X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = DecisionTree()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

print("accuracy:", accuracy_score(y_test, predictions))

accuracy: 0.9122807017543859
